In [147]:
import pandas as pd
import numpy as np
import sys
sys.path.insert(0, r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies')
# Run the other notebook
%run "C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\strategies_master.ipynb"
import ta

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [36]:
daily_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\stock_data_raw_daily.csv.gz', compression='gzip')
weekly_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\weekly_raw.csv.gz', compression='gzip')
monthly_data = pd.read_csv(r'C:\Users\40100171\OneDrive - Anheuser-Busch InBev\Desktop\Stock_market_strategies\data\raw\monthly_raw.csv.gz', compression='gzip')

In [37]:
weekly_model = weekly_data.copy()
weekly_model['c/o'] = weekly_model['Adj Close']/weekly_model['Open']
weekly_model['c/h'] = weekly_model['Adj Close']/weekly_model['High']
weekly_model['c/l'] = weekly_model['Adj Close']/weekly_model['Low']

weekly_model['Weekly_Return'] = weekly_model.groupby('Stock Name')['Adj Close'].pct_change(periods=1)
weekly_model['Weekly_Return'] = weekly_model.groupby('Stock Name')['Weekly_Return'].shift(-1)
weekly_model = weekly_model.loc[~(weekly_model['Weekly_Return'].isna()),]

weekly_model['postive return'] = weekly_model['Weekly_Return'].apply(lambda x: 1 if(x>0) else 0)

In [127]:
weekly_model['rsi_weekly'] = 0
for stock in weekly_model['Stock Name'].unique().tolist():
    weekly_model.loc[weekly_model['Stock Name']==stock,'rsi_weekly'] = ta.momentum.rsi(weekly_model.loc[weekly_model['Stock Name']==stock,'Close'])

weekly_model['rsi_weekly_shift_1'] = weekly_model.groupby('Stock Name')['rsi_weekly'].shift(1)
weekly_model['rsi_weekly_shift_2'] = weekly_model.groupby('Stock Name')['rsi_weekly'].shift(2)
weekly_model['rsi_weekly_shift_1'] = weekly_model['rsi_weekly']/weekly_model['rsi_weekly_shift_1']
weekly_model['rsi_weekly_shift_2'] = weekly_model['rsi_weekly']/weekly_model['rsi_weekly_shift_2']

weekly_model.replace([np.inf, -np.inf], np.nan, inplace=True)
weekly_model.dropna(inplace=True)
weekly_model.reset_index(drop=True, inplace=True)

In [176]:
Y = weekly_model['postive return']
#X = weekly_model[['c/o', 'c/h', 'c/l','rsi_weekly', 'rsi_weekly_shift_1', 'rsi_weekly_shift_2']]
X = weekly_model[['c/o', 'c/h', 'c/l','rsi_weekly']]

In [177]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_test)

In [178]:
# Feature Importance
importances = rf_classifier.feature_importances_
feature_names = X.columns

# Create a DataFrame for visualization
importance_df = pd.DataFrame({
    'Feature': feature_names,
    'Importance': importances
})

# Sort by importance
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

      Feature  Importance
3  rsi_weekly    0.263486
0         c/o    0.248659
2         c/l    0.247754
1         c/h    0.240100


In [179]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.5261


In [180]:
print("Classification Report:\n", classification_report(y_test, y_pred))

Classification Report:
               precision    recall  f1-score   support

           0       0.54      0.57      0.55     42997
           1       0.51      0.48      0.49     40056

    accuracy                           0.53     83053
   macro avg       0.52      0.52      0.52     83053
weighted avg       0.53      0.53      0.53     83053



In [181]:
# Print confusion matrix
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))

Confusion Matrix:
 [[24488 18509]
 [20848 19208]]
